In [ ]:
import os
import csv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, 
    BatchNormalization, concatenate, GlobalAveragePooling2D, Layer
)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import keras_tuner as kt

In [ ]:
basePath = "/Users/rathore/Documents/Logical_Rhythmm/Q3/"

In [ ]:
piece_mapping = {
    'black_king': 200, 'black_queen': 9, 
    'black_rook': 5, 'black_bishop': 3,
    'black_knight': 3, 'black_pawn': 1,
    
    'white_king': 200, 'white_queen': 9, 
    'white_rook': 5, 'white_bishop': 3,
    'white_knight': 3,'white_pawn': 1,
    
    'blank_': 0
}


piece_mapping_cnn = {
    'black_king': 10, 'black_queen': 9, 
    'black_rook': 7, 'black_bishop': 5, 
    'black_knight': 3, 'black_pawn': 1, 
    
    'white_king': 20, 'white_queen': 19, 
    'white_rook': 17, 'white_bishop': 15, 
    'white_knight': 13,'white_pawn': 11,
    
    'blank_': 0,
}

In [ ]:
def do(csv_file_path):
    print(csv_file_path)
    chessboard_matrix = []

    with open(csv_file_path, mode='r') as file:
        reader = csv.reader(file, delimiter='\t')
        for row in reader:
            arr = row[0].split(",");
            temp = []
            for j in range(8):
                temp.append(arr[j])
            chessboard_matrix.append(temp)
    return chessboard_matrix;

In [ ]:
train_csv_path = "/Users/rathore/Documents/Logical_Rhythmm/Q3/train.csv"
train_board_path = os.path.join("/Users/rathore/Documents/Logical_Rhythmm/Q3", "board_states")

train_data = pd.read_csv(train_csv_path)

def map_board_pieces(board_matrix):
    return [[piece_mapping.get(piece, 0) for piece in row] for row in board_matrix]

X_board_train = []

for _, row in train_data.iterrows():
    board_matrix = do(os.path.join(train_board_path, f"{row[0]}_board.csv"))  
    board_matrix = map_board_pieces(board_matrix)  
    board_matrix = np.array(board_matrix).astype(np.float32) 
    X_board_train.append(board_matrix)


X_board_train = np.array(X_board_train)
X_board_train = np.expand_dims(X_board_train, axis=-1)  

csv_file_path = "/Users/rathore/Documents/Logical_Rhythmm/Q3/train.csv"

train_data = pd.read_csv(csv_file_path, sep=',')

train_data['evaluation'] = train_data['evaluation'].map({'equal': 0, 'black': 1, 'white': 2})


ids = train_data['id'].values
y_train = train_data['evaluation'].values  

In [ ]:
!pip install pydot


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Dense, GlobalAveragePooling2D, concatenate
from tensorflow.keras.models import Model
import kerastuner as kt  # Ensure Keras Tuner is installed
from tensorflow.keras.utils import plot_model

train_csv_path = "/Users/rathore/Documents/Logical_Rhythmm/Q3/train.csv"
train_board_path = os.path.join("/Users/rathore/Documents/Logical_Rhythmm/Q3", "board_states")

train_data = pd.read_csv(train_csv_path)
train_data['evaluation'] = train_data['evaluation'].map({'equal': 0, 'black': 1, 'white': 2})
y_train = train_data['evaluation'].values


def map_board_pieces(board_matrix):
    return [[piece_mapping.get(piece, 0) for piece in row] for row in board_matrix]

def map_board_pieces_for_cnn(board_matrix, mapping):
    return [[mapping[piece] for piece in row] for row in board_matrix]

def map_board_pieces(board_matrix, mapping):
    return [[mapping[piece] for piece in row] for row in board_matrix]

def calculate_center_control(board_matrix):
    center_squares = [(3,3), (3,4), (4,3), (4,4)]
    control = {'black': 0, 'white': 0}
    for r, c in center_squares:
        piece = board_matrix[r][c]
        if "black" in piece:
            control['black'] += 1
        elif "white" in piece:
            control['white'] += 1
    return control

def calculate_piece_count_and_value(board_matrix):
    counts, values = {'black': 0, 'white': 0}, {'black': 0, 'white': 0}
    for row in board_matrix:
        for piece in row:
            if "black" in piece:
                counts['black'] += 1
                values['black'] += piece_mapping[piece]
            elif "white" in piece:
                counts['white'] += 1
                values['white'] += piece_mapping[piece]
    return counts, values

def calculate_enemy_base_pieces(board_matrix):
    black_in_white_half = sum(1 for r in range(4) for piece in board_matrix[r] if "black" in piece)
    white_in_black_half = sum(1 for r in range(4, 8) for piece in board_matrix[r] if "white" in piece)
    return black_in_white_half, white_in_black_half

def calculate_piece_safety(board_matrix):
    safety_scores = {'black': 0, 'white': 0}
    for i in range(8):
        for j in range(8):
            piece = board_matrix[i][j]
            if piece == 'blank_':
                continue
            neighbors = [(i+di, j+dj) for di in (-1, 0, 1) for dj in (-1, 0, 1) if (0 <= i+di < 8 and 0 <= j+dj < 8) and (di != 0 or dj != 0)]
            if any(board_matrix[x][y] != 'blank_' for x, y in neighbors):
                if 'black' in piece:
                    safety_scores['black'] += 1
                elif 'white' in piece:
                    safety_scores['white'] += 1
    return safety_scores

def calculate_piece_clustering(board_matrix):
    clustering_scores = {'black': 0, 'white': 0}
    for i in range(8):
        for j in range(8):
            piece = board_matrix[i][j]
            if piece == 'blank_':
                continue
            same_color_neighbors = [(i+di, j+dj) for di in (-1, 0, 1) for dj in (-1, 0, 1) if (0 <= i+di < 8 and 0 <= j+dj < 8) and (di != 0 or dj != 0) and board_matrix[i+di][j+dj] == piece]
            if 'black' in piece:
                clustering_scores['black'] += len(same_color_neighbors)
            elif 'white' in piece:
                clustering_scores['white'] += len(same_color_neighbors)
    return clustering_scores

def is_valid_move(board_matrix, i, j, x, y):
    if 0 <= x < 8 and 0 <= y < 8:
        return board_matrix[x][y] == 'blank_' or (board_matrix[x][y].startswith('black') and not board_matrix[i][j].startswith('black')) or (board_matrix[x][y].startswith('white') and not board_matrix[i][j].startswith('white'))
    return False

def calculate_knight_mobility(board_matrix, i, j, piece_color):
    knight_moves = [
        (i + 2, j + 1), (i + 2, j - 1), (i - 2, j + 1), (i - 2, j - 1),
        (i + 1, j + 2), (i + 1, j - 2), (i - 1, j + 2), (i - 1, j - 2)
    ]
    valid_moves = 0
    for x, y in knight_moves:
        if 0 <= x < 8 and 0 <= y < 8:
            if is_valid_move(board_matrix, i, j, x, y):
                valid_moves += 1
    return valid_moves

def calculate_bishop_mobility(board_matrix, i, j, piece_color):
    valid_moves = 0
    directions = [(-1, -1), (-1, 1), (1, -1), (1, 1)]  # diagonals
    for dx, dy in directions:
        x, y = i + dx, j + dy
        while 0 <= x < 8 and 0 <= y < 8:
            if board_matrix[x][y] == 'blank_':
                valid_moves += 1
            elif (piece_color in board_matrix[x][y] and board_matrix[x][y] != 'blank_'):
                break  
            else:
                valid_moves += 1  
                break
            x += dx
            y += dy
    return valid_moves

def calculate_rook_mobility(board_matrix, i, j, piece_color):
    valid_moves = 0
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)] 
    for dx, dy in directions:
        x, y = i + dx, j + dy
        while 0 <= x < 8 and 0 <= y < 8:
            if board_matrix[x][y] == 'blank_':
                valid_moves += 1
            elif (piece_color in board_matrix[x][y] and board_matrix[x][y] != 'blank_'):
                break  
            else:
                valid_moves += 1  
                break
            x += dx
            y += dy
    return valid_moves

def calculate_queen_mobility(board_matrix, i, j, piece_color):
    return calculate_bishop_mobility(board_matrix, i, j, piece_color) + calculate_rook_mobility(board_matrix, i, j, piece_color)

def calculate_king_mobility(board_matrix, i, j, piece_color):
    king_moves = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    valid_moves = 0
    for dx, dy in king_moves:
        x, y = i + dx, j + dy
        if is_valid_move(board_matrix, i, j, x, y):
            valid_moves += 1
    return valid_moves

def calculate_pawn_mobility(board_matrix, i, j, piece_color):
    valid_moves = 0
    direction = -1 if piece_color == 'white' else 1  
    start_row = 6 if piece_color == 'white' else 1  

    if board_matrix[i + direction][j] == 'blank_':
        valid_moves += 1  

        if i == start_row and board_matrix[i + 2 * direction][j] == 'blank_':
            valid_moves += 1 

    if 0 <= i + direction < 8:
        if j - 1 >= 0 and is_valid_move(board_matrix, i, j, i + direction, j - 1):
            valid_moves += 1
        if j + 1 < 8 and is_valid_move(board_matrix, i, j, i + direction, j + 1):
            valid_moves += 1

    return valid_moves

def calculate_piece_mobility(board_matrix):
    mobility_scores = {'black': 0, 'white': 0}

    for i in range(8):
        for j in range(8):
            piece = board_matrix[i][j]
            if piece == 'blank_':
                continue
            piece_color = 'black' if 'black' in piece else 'white'

            if 'knight' in piece:
                mobility_scores[piece_color] += calculate_knight_mobility(board_matrix, i, j, piece_color)
            elif 'bishop' in piece:
                mobility_scores[piece_color] += calculate_bishop_mobility(board_matrix, i, j, piece_color)
            elif 'rook' in piece:
                mobility_scores[piece_color] += calculate_rook_mobility(board_matrix, i, j, piece_color)
            elif 'queen' in piece:
                mobility_scores[piece_color] += calculate_queen_mobility(board_matrix, i, j, piece_color)
            elif 'king' in piece:
                mobility_scores[piece_color] += calculate_king_mobility(board_matrix, i, j, piece_color)
            elif 'pawn' in piece:
                mobility_scores[piece_color] += calculate_pawn_mobility(board_matrix, i, j, piece_color)

    return mobility_scores


def calculate_pawn_structure(board_matrix):
    pawn_structure = {
        'black': {'count': 0, 'doubled': 0, 'isolated': 0},
        'white': {'count': 0, 'doubled': 0, 'isolated': 0}
    }
    
    for color in ['black', 'white']:
        pawns_in_columns = [0] * 8  # Track number of pawns in each column for each color
        for i in range(8):
            for j in range(8):
                piece = board_matrix[i][j]
                if f'{color}_pawn' in piece:
                    pawn_structure[color]['count'] += 1
                    pawns_in_columns[j] += 1

        for j in range(8):
            if pawns_in_columns[j] > 1:
                pawn_structure[color]['doubled'] += pawns_in_columns[j] - 1  # All except the first are "doubled"
            if (j == 0 or pawns_in_columns[j - 1] == 0) and (j == 7 or pawns_in_columns[j + 1] == 0):
                pawn_structure[color]['isolated'] += pawns_in_columns[j]  # Isolated if no adjacent column has pawns

    return pawn_structure

def calculate_king_safety(board_matrix):
    king_safety = {
        'black': {'count': 0, 'pawn_protection': 0, 'center_exposure': 0},
        'white': {'count': 0, 'pawn_protection': 0, 'center_exposure': 0}
    }
    center_squares = {(3, 3), (3, 4), (4, 3), (4, 4)}
    directions = [(-1, -1), (-1, 1), (1, -1), (1, 1), (0, -1), (0, 1), (-1, 0), (1, 0)]

    for color in ['black', 'white']:
        for i in range(8):
            for j in range(8):
                piece = board_matrix[i][j]
                if f'{color}_king' in piece:
                    king_safety[color]['count'] += 1
                    if (i, j) in center_squares:
                        king_safety[color]['center_exposure'] += 1 
                    for di, dj in directions:
                        ni, nj = i + di, j + dj
                        if 0 <= ni < 8 and 0 <= nj < 8 and f'{color}_pawn' in board_matrix[ni][nj]:
                            king_safety[color]['pawn_protection'] += 1
    
    return king_safety


def extract_features(board_matrix):
    center_control = calculate_center_control(board_matrix)
    piece_counts, piece_values = calculate_piece_count_and_value(board_matrix)
    black_in_white_half, white_in_black_half = calculate_enemy_base_pieces(board_matrix)
    piece_safety = calculate_piece_safety(board_matrix)
    piece_clustering = calculate_piece_clustering(board_matrix)
    piece_mobility = calculate_piece_mobility(board_matrix)
    pawn_structure = calculate_pawn_structure(board_matrix)
    king_safety = calculate_king_safety(board_matrix)
    
    return [
        center_control['black'], center_control['white'],
        piece_counts['black'], piece_counts['white'],
        piece_values['black'], piece_values['white'],
        black_in_white_half, white_in_black_half,
        piece_safety['black'], piece_safety['white'],
        piece_clustering['black'], piece_clustering['white'],
        piece_mobility['black'], piece_mobility['white'],
        pawn_structure['black']['count'], pawn_structure['white']['count'],
        pawn_structure['black']['doubled'], pawn_structure['white']['doubled'],
        pawn_structure['black']['isolated'], pawn_structure['white']['isolated'],
        king_safety['black']['count'], king_safety['white']['count'],
        king_safety['black']['pawn_protection'], king_safety['white']['pawn_protection'],
        king_safety['black']['center_exposure'], king_safety['white']['center_exposure']
    ]


X_board_train = []
X_features_train = []

for _, row in train_data.iterrows():
    board_matrix_df = pd.read_csv(os.path.join(train_board_path, f"{row['id']}_board.csv"), header=None)
    board_matrix = board_matrix_df.values  

    board_matrix_int = map_board_pieces(board_matrix, piece_mapping_cnn)
    X_board_train.append(np.array(board_matrix_int).astype(np.float32))

    additional_features = extract_features(board_matrix)
    X_features_train.append(additional_features)

X_board_train = np.expand_dims(np.array(X_board_train), axis=-1)  # Shape: (num_samples, 8, 8, 1)
X_features_train = np.array(X_features_train)

X_board_train, X_board_val, X_features_train, X_features_val, y_train, y_val = train_test_split(
    X_board_train, X_features_train, y_train, test_size=0.2, random_state=21
)

class StabilizedSoftmax(Layer):
    def call(self, inputs):
        logits = inputs - tf.reduce_max(inputs, axis=-1, keepdims=True)  
        return tf.nn.softmax(logits, axis=-1) 

def build_model(hp):
    input_board = Input(shape=(8, 8, 1), name="board_input")
    x = Conv2D(hp.Choice('conv_1_filters', [32, 64, 128]), (3, 3), activation='relu', padding="same")(input_board)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(hp.Choice('dropout_1', [0.2, 0.3]))(x)

    x = Conv2D(hp.Choice('conv_2_filters', [64, 128, 256]), (3, 3), activation='relu', padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(hp.Choice('dropout_2', [0.3, 0.4]))(x)

    x = GlobalAveragePooling2D()(x)

    input_features = Input(shape=(26,), name="additional_features")  
    y = Dense(32, activation='relu')(input_features)
    y = BatchNormalization()(y)
    y = Dropout(0.3)(y)

    y = Dense(64, activation='relu')(y)
    y = BatchNormalization()(y)
    y = Dropout(0.3)(y)
    
    y = Dense(128, activation='relu')(y)
    y = BatchNormalization()(y)
    y = Dropout(0.4)(y)

    combined = concatenate([x, y])

    z = Dense(hp.Choice('dense_units_1', [128, 256, 512]), activation='relu')(combined)
    z = BatchNormalization()(z)
    z = Dropout(0.4)(z)
    
    z = Dense(hp.Choice('dense_units_2', [128, 256, 512]), activation='relu')(z)
    z = BatchNormalization()(z)
    z = Dropout(0.4)(z)

    logits = Dense(3)(z) 
    outputs = StabilizedSoftmax()(logits)

    model = Model(inputs=[input_board, input_features], outputs=outputs)

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4]),
        decay_steps=10000,
        decay_rate=0.9,
        staircase=True
    )
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


tuner_unique_2 = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='/Users/rathore/tuner_logs_unique_5',
    project_name='chess_model_unique'  
)

tuner_unique_2.search([X_board_train, X_features_train], y_train, 
                    validation_data=([X_board_val, X_features_val], y_val), epochs=10)

best_model_unique_2 = tuner_unique_2.get_best_models(1)[0]

best_model_unique_2.evaluate([X_board_val, X_features_val], y_val)

import tensorflow as tf
from tensorflow.keras.utils import plot_model

try:
    hp = kt.HyperParameters()
    model = build_model(hp)
    
    plot_model(model, to_file='model_structure.png', show_shapes=True, show_layer_names=True, dpi=96)
    
    from IPython.display import Image
    display(Image('model_structure.png'))
    
except Exception as e:
    print("An error occurred:", e)
    print("Ensure 'pydot' and 'graphviz' are installed and accessible.")


In [ ]:
best_model_unique_2 = tuner_unique_2.get_best_models(1)[0]
best_hyperparameters = tuner_unique_2.get_best_hyperparameters(num_trials=1)[0]

loss, accuracy = best_model_unique_2.evaluate([X_board_val, X_features_val], y_val)
print(f"Best Model Validation Accuracy: {accuracy * 100:.2f}%")
print("Best Hyperparameters:", best_hyperparameters.values)

In [ ]:
test_board_path = "/Users/rathore/Documents/Logical_Rhythmm/Q3/board_states_test"

test_results = []

for file_name in os.listdir(test_board_path):
    if file_name.endswith("_board.csv"):
        game_id = file_name.split("_board")[0]

        board_matrix_df = pd.read_csv(os.path.join(test_board_path, file_name), header=None)
        board_matrix = board_matrix_df.values  # Assuming it's in string format

        center_control = calculate_center_control(board_matrix)
        piece_counts, piece_values = calculate_piece_count_and_value(board_matrix)
        black_in_white_half, white_in_black_half = calculate_enemy_base_pieces(board_matrix)
        piece_safety = calculate_piece_safety(board_matrix)
        piece_clustering = calculate_piece_clustering(board_matrix)
        piece_mobility = calculate_piece_mobility(board_matrix)
        pawn_structure = calculate_pawn_structure(board_matrix)
        king_safety = calculate_king_safety(board_matrix)
        
        additional_features = [
            center_control['black'], center_control['white'],
            piece_counts['black'], piece_counts['white'],
            piece_values['black'], piece_values['white'],
            black_in_white_half, white_in_black_half,
            piece_safety['black'], piece_safety['white'],
            piece_clustering['black'], piece_clustering['white'],
            piece_mobility['black'], piece_mobility['white'],
            pawn_structure['black'], pawn_structure['white'],
            king_safety['black'], king_safety['white']
        ]

        board_matrix_int = map_board_pieces(board_matrix)

        board_input = np.expand_dims(np.array(board_matrix_int).astype(np.float32), axis=(0, -1))  # Shape: (1, 8, 8, 1)
        features_input = np.array([additional_features])  # Shape: (1, 8)

        prediction = best_model_unique.predict([board_input, features_input])
        predicted_label = np.argmax(prediction)

        evaluation_mapping = {0: "equal", 1: "black", 2: "white"}
        evaluation = evaluation_mapping[predicted_label]

        test_results.append({"id": game_id, "evaluation": evaluation})



In [ ]:
output_df = pd.DataFrame(test_results)
output_df.to_csv("test_predictions_2.csv", index=False)
print("Predictions saved to test_predictions_2.csv")



In [ ]:
output_df = pd.DataFrame(test_results)
output_df.to_csv("test_predictions_2.csv", index=False)
print("Predictions saved to test_predictions_2.csv")
